In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import math
import time
from tqdm import tqdm

In [2]:
BASE_URL = "https://www.kjnala.com"
search_url = f'{BASE_URL}/board/index.jsp?code=bbs018&s_category=13126&s_keyword='
page = requests.get(search_url)
soup = BeautifulSoup(page.content, 'html.parser')

num = soup.select('td.pc_line')[0].text
end_num = math.ceil(int(num) / 10)

df = pd.read_csv('./로열종목.csv')
stop_date = df['date'].iloc[0] 

In [3]:
sp_dates =[]
def get_items(date, link):
    search_url = f'{BASE_URL}{link}'
    page = requests.get(search_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    items = soup.select('div.contArea > p')
    item_list = [date]
    
    if len(items) == 1:
        item = re.sub('<p>', '', str(items[0]))
        item = re.sub('</p>', '', item)
        item = re.sub('<br/>', '', item)
        item = item.split("-")
        for i in item:
            if i !='' and "※" not in item and item[0].isdigit() == False:
                item_list.append(i)
        item_list
    else:
    
        if len(str(items)) > 200:
            sp_dates.append(date)
            for i in range(len(items)):
                item = items[i].text.split(' ', 1)[0]
                item = item.replace(u'\xa0', u'')
                item = re.sub('-', '', item) 
                if item !='' and "※" not in item and item[0].isdigit() == False:
                    item_list.append(item)
        else: 
            for i in range(len(items)):
                item = re.sub('-', '', items[i].text) 
                item = re.sub(' ', '', item) 
                item = item.replace(u'\xa0', u'')
                if item !='' and "※" not in item and item[0].isdigit() == False:
                    item_list.append(item)

    for i in range(7 - len(item_list)):
        item_list.append("None")
            
    return item_list

In [4]:
data = pd.DataFrame()

def get_link(page):
    search_url = f'{BASE_URL}/board/index.jsp?code=bbs017&page={page}'
    page = requests.get(search_url)
    soup = BeautifulSoup(page.content, 'html.parser')
        
    global stop_reached  # Use global flag
    
    nname = soup.select('p.subject > a')
    
    for i in range(len(nname)):
        title = re.sub('\s+', '', nname[i].text)
        number = re.findall("\d+", title)
        date = '-'.join(number)[0:10]
        link = re.sub('code=bbs017&', '', nname[i]['href'])
        
        if date == stop_date:
            stop_reached = True  # Set the flag to indicate stop_date is reached
            return data  # Exit the function
        
        data.loc[len(data)] = get_items(date, link)
    return data

In [5]:
data = pd.DataFrame(columns=['date', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6'])
sp_dates = []
for i in tqdm(range(1, end_num+1)): 
    data = get_link(i)
    if stop_reached:  # Check if stop_date is reached inside get_link
        break  # Break out of the loop if the stop_date is reached
    time.sleep(2)

  0%|          | 0/13 [00:00<?, ?it/s]


In [6]:
df_concatenated = pd.concat([data, df], ignore_index=True)
pd.set_option('display.max_rows', 5000)
display(df_concatenated)

,date,item1,item2,item3,item4,item5,item6
0,2024-01-31,솔본,아센디오,엘컴텍,유니온커뮤니티,포메탈,랩지노믹스
1,2024-01-29,엑세스바이오,액션스퀘어,바이넥스,필옵틱스,피엠티,HLB글로벌
2,2024-01-26,세코닉스,신화인터텍,에스엔유,가온그룹,LB세미콘,LB루셈
3,2024-01-24,키다리스튜디오,가온그룹,메가엠디,상신전자,조이시티,노랑풍선
4,2024-01-22,한컴위드,티피씨글로벌,로체시스템즈,엠투아이,탑엔지니어링,플래티어
5,2024-01-19,삼진엘앤디,포바이포,신송홀딩스,메가엠디,이랜텍,한성크린텍
6,2024-01-17,와이어블,일승,신화인터텍,삼륭물산,삼영전자,포바이포
7,2024-01-15,코위버,솔본,일승,신송홀딩스,와이투솔루션,케이엘넷
8,2024-01-12,삼일씨엔에스,TPC,세코닉스,신송홀딩스,동방,에스피시스템스
9,2024-01-10,신화인터텍,한컴위드,엔피,라이온켐텍,KC코트렐,대영포장


In [7]:
df_concatenated.to_csv("로열종목.csv", index=False, encoding="utf-8-sig")

: 